# StockPulse: A Sentiment-Based Stock Forecasting Tool

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
from scipy.stats import pearsonr
from sklearn.decomposition import TruncatedSVD

In [ ]:
import nltk
import re
import string
from sklearn.naive_bayes import MultinomialNB

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
stock_tweets = pd.read_csv("/content/drive/MyDrive/Classroom/Stock_Tweets/stock_tweets.csv")
yahoo_finance = pd.read_csv("/content/drive/MyDrive/Classroom/Stock_Tweets/stock_yfinance_data.csv")

print(stock_tweets.head())

                        Date  \
0  2022-09-29 23:41:16+00:00   
1  2022-09-29 23:24:43+00:00   
2  2022-09-29 23:18:08+00:00   
3  2022-09-29 22:40:07+00:00   
4  2022-09-29 22:27:05+00:00   

                                               Tweet Stock Name Company Name  
0  Mainstream media has done an amazing job at br...       TSLA  Tesla, Inc.  
1  Tesla delivery estimates are at around 364k fr...       TSLA  Tesla, Inc.  
2  3/ Even if I include 63.0M unvested RSUs as of...       TSLA  Tesla, Inc.  
3  @RealDanODowd @WholeMarsBlog @Tesla Hahaha why...       TSLA  Tesla, Inc.  
4  @RealDanODowd @Tesla Stop trying to kill kids,...       TSLA  Tesla, Inc.  


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


df = stock_tweets[:10]

# Data preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

# Apply the preprocessing function to the 'text' column
df['Tweet'] = df['Tweet'].apply(preprocess_text)

# Tokenization and removing stop words
stop_words = set(stopwords.words('english'))

# Apply tokenization to the 'text' column
df['tokens'] = df['Tweet'].apply(lambda x: [word for word in word_tokenize(x) if word.isalpha() and word not in stop_words])

# TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust max_features as needed
X_tfidf = vectorizer.fit_transform(df['Tweet'])

# Train the classifier on the entire dataset
classifier = MultinomialNB()

classifier.fit(X_tfidf, df['Tweet'])

# Make predictions for the entire dataset
df['sentiment'] = classifier.predict(X_tfidf)

# Save the DataFrame back to the Excel file with the new 'sentiment' column
df.to_excel('your_updated_dataset.xlsx', index=False)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
<ipython-input-6-a0a5000daa02>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweet'] = df['Tweet'].apply(preprocess_text)
<ipython-input-6-a0a5000daa02>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['Tweet'].apply(lambda x: [word for word in word_tokenize(x) if wor

In [ ]:
import pandas as pd
from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis")

df = stock_tweets[:50]

df['sentiment'] = df['Tweet'].apply(lambda x: sentiment_analysis(x)[0]['label'])

sentiment_mapping = {'POSITIVE': 1, 'NEUTRAL': 0, 'NEGATIVE': -1}
df['sentiment'] = df['sentiment'].map(sentiment_mapping)

# Print or save the DataFrame with sentiment predictions
print(df)
df.to_csv('your_updated_dataset.csv', index=False)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


                         Date  \
0   2022-09-29 23:41:16+00:00   
1   2022-09-29 23:24:43+00:00   
2   2022-09-29 23:18:08+00:00   
3   2022-09-29 22:40:07+00:00   
4   2022-09-29 22:27:05+00:00   
5   2022-09-29 22:25:53+00:00   
6   2022-09-29 22:24:22+00:00   
7   2022-09-29 22:23:54+00:00   
8   2022-09-29 22:23:28+00:00   
9   2022-09-29 22:15:01+00:00   
10  2022-09-29 21:58:00+00:00   
11  2022-09-29 21:33:05+00:00   
12  2022-09-29 21:11:30+00:00   
13  2022-09-29 21:00:25+00:00   
14  2022-09-29 20:15:41+00:00   
15  2022-09-29 19:38:29+00:00   
16  2022-09-29 19:36:59+00:00   
17  2022-09-29 19:27:54+00:00   
18  2022-09-29 19:24:50+00:00   
19  2022-09-29 19:17:48+00:00   
20  2022-09-29 19:12:32+00:00   
21  2022-09-29 19:10:37+00:00   
22  2022-09-29 19:08:03+00:00   
23  2022-09-29 19:01:22+00:00   
24  2022-09-29 18:48:36+00:00   
25  2022-09-29 18:46:07+00:00   
26  2022-09-29 18:43:55+00:00   
27  2022-09-29 18:36:51+00:00   
28  2022-09-29 18:23:20+00:00   
29  2022-0

<ipython-input-7-83d58ed5b593>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['Tweet'].apply(lambda x: sentiment_analysis(x)[0]['label'])
<ipython-input-7-83d58ed5b593>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].map(sentiment_mapping)


In [ ]:
import pandas as pd
grouped_df = df.groupby('Stock Name')['sentiment'].sum().reset_index()
print(grouped_df)

  Stock Name  sentiment
0       TSLA        -20


In [ ]:
# Example: Calculate 5-day moving average sentiment score
df['MA5_sentiment'] = df['sentiment'].rolling(window=5).mean()

# Example: Count the number of tweets per day
df['Tweet_Count'] = df.groupby('Date')['Tweet'].transform('count')

<ipython-input-9-0f9862f45d57>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MA5_sentiment'] = df['sentiment'].rolling(window=5).mean()
<ipython-input-9-0f9862f45d57>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tweet_Count'] = df.groupby('Date')['Tweet'].transform('count')


In [ ]:
df

,Date,Tweet,Stock Name,Company Name,sentiment,MA5_sentiment,Tweet_Count
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",1,NaN,1
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",-1,NaN,1
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.",-1,NaN,1
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",-1,NaN,1
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.",-1,-0.6,1
5,2022-09-29 22:25:53+00:00,@RealDanODowd @Tesla This is you https://t.co/...,TSLA,"Tesla, Inc.",-1,-1.0,1
6,2022-09-29 22:24:22+00:00,For years @WholeMarsBlog viciously silenced @T...,TSLA,"Tesla, Inc.",1,-0.6,1
7,2022-09-29 22:23:54+00:00,$NIO just because I'm down money doesn't mean ...,TSLA,"Tesla, Inc.",1,-0.2,1
8,2022-09-29 22:23:28+00:00,50 likes for some $SPY $TSLA charts to study!\...,TSLA,"Tesla, Inc.",1,0.2,1
9,2022-09-29 22:15:01+00:00,@MrJames__321 @KellyRoofing @TeslaSolar @elonm...,TSLA,"Tesla, Inc.",-1,0.2,1


In [ ]:
df2 = yahoo_finance[:50]

In [ ]:
# Calculate daily stock price percentage change
df2['Price_Change'] = df2['Close'].pct_change()

# Calculate the 30-day rolling standard deviation of stock prices
df2['Volatility'] = df2['Close'].rolling(window=30).std()

In [ ]:
df2

In [ ]:

# Convert the 'DateTime' column to a pandas datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Create separate columns for date and time
df['DATE'] = df['Date'].dt.date
# df['TIME'] = df['Date'].dt.time



In [ ]:
df.head()

In [ ]:
df2 = df2.rename(columns={'Date': 'DATE'})

In [ ]:
df3 = pd.merge(df2, df, how='left', on='DATE')

In [ ]:
df3.head()

In [ ]:
# df['Lagged_Sentiment'] = df['sentiment'].shift(1)

In [ ]:
df3['Day_of_Week'] = df3['Date'].dt.dayofweek

In [ ]:

# Drop the original 'date_column' and 'time_column' if needed
df3 = df3.drop(columns=['DATE'])


df3.head()

In [ ]:
df3['timestamp'] = df3['Date'].astype(int) // 10**9
df3 = df3.drop(columns=['Date'])

In [ ]:
df3.head()

TESTING AND TRAINING DATA SPLIT

In [ ]:
selected_columns_range = df3.loc[:, 'Open':'Volume'].columns.tolist() + df3.loc[:, 'Price_Change':'Volatility'].columns.tolist() + df3.loc[:, 'sentiment':'Tweet_Count'].columns.tolist() + ['timestamp']
df4 = df3[selected_columns_range].copy()

df4.fillna(0, inplace=True)

In [ ]:
df4.head()

In [ ]:
# Load the dataset
data = stock_tweets

# Preprocess the tweet text
nltk.download('stopwords')
stop_words = set(nltk.corpus.stopwords.words('english'))

def preprocess_text(text):
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    return ' '.join(words)

data['Processed_Tweet'] = data['Tweet'].apply(preprocess_text)

# User input
user_input = "I'm excited about Tesla, it has great potential."
user_input = preprocess_text(user_input)

# Sentiment analysis
def get_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity

data['Sentiment'] = data['Processed_Tweet'].apply(get_sentiment)

grouped_stocks = data.groupby('Company Name')['Sentiment'].sum().reset_index()

# Create a TF-IDF vectorizer for the processed tweets
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(grouped_stocks['Company Name'])

# Calculate cosine similarity between user input and tweets
user_tfidf = tfidf_vectorizer.transform([user_input])
cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix)

# Add the similarity scores to the DataFrame
grouped_stocks['Similarity'] = cosine_similarities.flatten()

# Sort stocks based on similarity scores in descending order
recommended_stocks = grouped_stocks.sort_values(by='Similarity', ascending=False)['Company Name'][:1]

recommended_stocks

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


18    Tesla, Inc.
Name: Company Name, dtype: object

In [ ]:
print(tfidf_matrix[0,24])

0.0


In [ ]:
print(user_tfidf)

  (0, 38)	1.0


In [ ]:
data

,Date,Tweet,Stock Name,Company Name,Processed_Tweet,Sentiment
0,2022-09-29 23:41:16+00:00,Mainstream media has done an amazing job at br...,TSLA,"Tesla, Inc.",Mainstream media done amazing job brainwashing...,0.600000
1,2022-09-29 23:24:43+00:00,Tesla delivery estimates are at around 364k fr...,TSLA,"Tesla, Inc.",Tesla delivery estimates around 364k analysts....,0.000000
2,2022-09-29 23:18:08+00:00,3/ Even if I include 63.0M unvested RSUs as of...,TSLA,"Tesla, Inc.","3/ Even include 63.0M unvested RSUs 6/30, addi...",0.018182
3,2022-09-29 22:40:07+00:00,@RealDanODowd @WholeMarsBlog @Tesla Hahaha why...,TSLA,"Tesla, Inc.",@RealDanODowd @WholeMarsBlog @Tesla Hahaha sti...,0.079167
4,2022-09-29 22:27:05+00:00,"@RealDanODowd @Tesla Stop trying to kill kids,...",TSLA,"Tesla, Inc.","@RealDanODowd @Tesla Stop trying kill kids, sa...",-0.200000
...,...,...,...,...,...,...
80788,2021-10-07 17:11:57+00:00,Some of the fastest growing tech stocks on the...,XPEV,XPeng Inc.,fastest growing tech stocks market. Mix #Finte...,0.000000
80789,2021-10-04 17:05:59+00:00,"With earnings on the horizon, here is a quick ...",XPEV,XPeng Inc.,"earnings horizon, quick snapshot largest incre...",0.111111
80790,2021-10-01 04:43:41+00:00,Our record delivery results are a testimony of...,XPEV,XPeng Inc.,record delivery results testimony unwavering p...,0.000000
80791,2021-10-01 00:03:32+00:00,"We delivered 10,412 Smart EVs in Sep 2021, rea...",XPEV,XPeng Inc.,"delivered 10,412 Smart EVs Sep 2021, reaching ...",0.107143


In [ ]:
grouped_stocks

,Stock Name,Sentiment
0,AAPL,428.689832
1,AMD,313.428019
2,AMZN,357.504355
3,BA,46.689264
4,BX,8.525335
5,COST,45.287499
6,CRM,32.874471
7,DIS,54.844157
8,ENPH,30.226810
9,F,3.662734


In [ ]:
# Calculate Pearson correlation between user input and tweets
def pearson_correlation(user_tfidf, tweet_tfidf):
    return pearsonr(user_tfidf.toarray().flatten(), tweet_tfidf.toarray().flatten())[0]

pearson_correlations = [pearson_correlation(user_tfidf, tweet_tfidf) for tweet_tfidf in tfidf_matrix]
# print(pearson_correlations)

# Recommendation based on Pearson correlation
def recommend_stocks_pearson(correlation_scores, threshold=0.5):
    relevant_indices = [i for i, score in enumerate(correlation_scores) if score > threshold]
    recommended_stocks = grouped_stocks['Company Name'][relevant_indices].unique()
    return recommended_stocks

# Get recommended stocks based on Pearson correlation
recommended_stocks_pearson = recommend_stocks_pearson(pearson_correlations, threshold=0)

print("Recommended stocks based on Pearson Correlation:", recommended_stocks_pearson)

Recommended stocks based on Pearson Correlation: ['Tesla, Inc.']


In [ ]:
# Recommendation based on cosine similarity
def recommend_stocks_cosine(similarity_scores, threshold=0.5):
    relevant_indices = [i for i, score in enumerate(similarity_scores) if score.any() > threshold]
    recommended_stocks = data['Stock Name'][relevant_indices].unique()
    return recommended_stocks

# Get recommended stocks based on cosine similarity
recommended_stocks_cosine = recommend_stocks_cosine(cosine_similarities, threshold=0.5)

print("Recommended stocks based on Cosine Similarity:", recommended_stocks_cosine)

Recommended stocks based on Cosine Similarity: ['TSLA']


In [ ]:
# Matrix Factorization (SVD) recommendation
def recommend_stocks_svd(tfidf_matrix, user_tfidf, threshold=0.5, num_components=5):
    # Perform Truncated SVD on the TF-IDF matrix
    svd = TruncatedSVD(n_components=num_components)
    tfidf_matrix_svd = svd.fit_transform(tfidf_matrix)
    user_tfidf_svd = svd.transform(user_tfidf)

    # Calculate cosine similarity between user input and tweets in SVD space
    cosine_similarities_svd = cosine_similarity(user_tfidf_svd, tfidf_matrix_svd)

    relevant_indices = [i for i, score in enumerate(cosine_similarities_svd) if score.any() > threshold]
    recommended_stocks = grouped_stocks['Company Name'][relevant_indices].unique()
    return recommended_stocks

# Get recommended stocks based on SVD similarity
recommended_stocks_svd = recommend_stocks_svd(tfidf_matrix, user_tfidf, threshold=0.2, num_components=50)

print("Recommended stocks based on SVD Similarity:", recommended_stocks_svd)

ValueError: ignored

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Assuming 'stock_score_matrix' is your DataFrame where rows are stocks and columns are features or scores.

# Calculate the cosine similarity between stocks
item_similarity_matrix = cosine_similarity(df4, dense_output=False)

# Recommend stocks for a specific stock
stock_id = 1
stock_scores = df4.loc[stock_id].values.reshape(1, -1)
stock_recommendations = cosine_similarity(stock_scores, item_similarity_matrix).flatten()

# Sort recommendations and get top N
top_recommendations = stock_recommendations.argsort()[::-1][:N]

ValueError: ignored

In [ ]:
from sklearn.model_selection import train_test_split

X = df4.drop(columns=['Volatility'])
y = df4['Volatility']

random_seed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)


MODEL SELECTION

IMPORT LIABRARIES

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [ ]:

X = df4['sentiment']
y = df4['Volatility']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

svm_classifier = SVC(kernel='linear', C=1.0, random_state=42)

# X = X.reshape(-1,1)
# y = y.reshape(-1,1)

svm_classifier.fit(X_train, y_train)

svm_predictions = svm_classifier.predict(X_test)

svm_accuracy = accuracy_score(y_test, svm_predictions)
print("Support Vector Machine Accuracy:", svm_accuracy)


AttributeError: ignored

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Assuming you have true labels y_true and predicted labels y_pred
y_true = y_train #[1, 0, 1, 0, 1, 0, 0, 1, 1, 1]
y_pred = y_test #[1, 0, 1, 1, 1, 0, 1, 0, 1, 0]

# Accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Precision
precision = precision_score(y_true, y_pred)
print("Precision:", precision)

# Recall
recall = recall_score(y_true, y_pred)
print("Recall:", recall)

# F1-score
f1 = f1_score(y_true, y_pred)
print("F1 Score:", f1)

# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


REGRESSION EVALUATION METRICS

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# y_true = [100, 110, 105, 120, 130, 115]
# y_pred = [98, 112, 103, 122, 128, 118]

# Mean Absolute Error (MAE)
mae = mean_absolute_error(y_true, y_pred)
print("MAE:", mae)

# Root Mean Squared Error (RMSE)
rmse = mean_squared_error(y_true, y_pred, squared=False)
print("RMSE:", rmse)

# R-squared (R^2)
r2 = r2_score(y_true, y_pred)
print("R-squared:", r2)


TESTING AND VALIDATION

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification Report
class_report = classification_report(y_test, y_pred)
print("Classification Report:")
print(class_report)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


ValueError: ignored

STOCK RANKING

Step 1: Calculate Stock Scores

In [ ]:
import pandas as pd

data = {
    'Stock_Ticker': ['AAPL', 'TSLA', 'GOOGL', 'MSFT'],
    'Sentiment_Score': [0.8, 0.6, 0.5, 0.7]
}

stock_scores_df = pd.DataFrame(data)


Step 2: Rank the Stocks

In [ ]:
# Sort stocks by sentiment score in descending order
ranked_stocks = stock_scores_df.sort_values(by='Sentiment_Score', ascending=False)


Step 3: View the Ranked Stocks

In [ ]:
print("Ranked Stocks:")
print(ranked_stocks)


Ranked Stocks:
  Stock_Ticker  Sentiment_Score
0         AAPL              0.8
3         MSFT              0.7
1         TSLA              0.6
2        GOOGL              0.5


BACK TESTING

In [ ]:
import pandas as pd

data = {
    'Date': ['2023-01-01', '2023-01-02', '2023-01-03', ...],
    'Stock_Price_AAPL': [150.0, 151.0, 152.5, ...],
    'Stock_Price_TSLA': [900.0, 910.0, 920.0, ...],
    'Sentiment_Score_AAPL': [0.8, 0.7, 0.9, ...],
    'Sentiment_Score_TSLA': [0.6, 0.5, 0.7, ...],
    'Signal_AAPL': [1, 0, -1, ...],  # Buy, Hold, Sell
    'Signal_TSLA': [0, 1, -1, ...],
}

historical_data = pd.DataFrame(data)


In [ ]:
# Initialize portfolio and holdings
portfolio = {}
holdings = {}
cash = 10000  # Initial

def backtest_strategy(data):
    for index, row in data.iterrows():
        date = row['Date']
        for stock in ['AAPL', 'TSLA']:
            signal = row[f'Signal_{stock}']
            price = row[f'Stock_Price_{stock}']

            if stock not in portfolio:
                portfolio[stock] = 0

            if signal == 1:
                # Buy
                shares_to_buy = int(cash / price)
                portfolio[stock] += shares_to_buy
                cash -= shares_to_buy * price
            elif signal == -1:
                # Sell all holdings
                cash += portfolio[stock] * price
                portfolio[stock] = 0

            holdings[date] = {stock: portfolio[stock], 'Cash': cash}

# Perform backtesting
backtest_strategy(historical_data)
